In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

from sklearn.preprocessing import PowerTransformer, MinMaxScaler


c:\Users\nuno\Miniconda3\envs\ml\lib\site-packages\statsforecast\core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#load
data_raw = pd.read_excel('dataset_cost_3.xlsx', sheet_name='data')
data = data_raw.fillna(0).copy()
#replace_negatives = lambda x: 0 if x < 0 else x
#data['value'] = data['value'].map(replace_negatives)
#data['value'] = data['value']/100
data.head()

,04 FISCAL MONTH,id,Contact Center OH Expense,Phone Support Expense,Non-Phone Expense,E-Support Expenses,Contact Center Residual,Other Warranty Expense,Architecture and Change,General Management Costs,...,GBU OH,Reimbursable Quality Issue,Unit Impacts,Adjustment Impacts,Delivery,Supply Chain,Other Repair Cost,Delivery OH,Supply Chain OH,Level 2 Support
0,201611,North America Market + AMS Other_PL4L,0.0,3054.321877,2199.701673,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,1247.37,-12958.35,4851.032028,182641.015625,-28279.235229,0.0,0.0,0.0
1,201611,North America Market + AMS Other_PL6A,0.0,8980.690000,3867.290000,0.0,0.0,0.0,0.0,0.0,...,730.115654,0.0,-7929.57,-30674.58,23.111818,310.248169,0.000000,0.0,0.0,0.0
2,201611,North America Market + AMS Other_PLGT,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.00,0.000000,0.000000,255.000000,0.0,0.0,0.0
3,201611,North America Market + AMS Other_PLMA,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,-38304.67,-35612.32,1710.573608,188662.280273,0.000000,0.0,0.0,0.0
4,201611,North America Market + AMS Other_PLMC,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.00,4.753579,-544.243591,0.000000,0.0,0.0,0.0


In [6]:
data = pd.melt(frame=data, id_vars=['04 FISCAL MONTH', 'id'], value_name='y', var_name='line_cost')

In [7]:
#load
data_reg_raw = pd.read_excel('dataset_reg_3.xlsx', sheet_name='data')
data_reg = data_reg_raw.fillna(0)
data_reg.head()

,04 FISCAL MONTH,id,Net Revenues,Warranty Units Adjusted,WIB,Total Number of Calls,Total Number of Repair Events,AIR (Avg)
0,201611,North America Market + AMS Other_PL4L,198428.84872,166.0,1163.0,68.630716,0.0,0.0
1,201611,Latin America Market_PL4L,36976.62628,61.0,414.0,46.718345,0.0,0.0
2,201611,Southern Europe Market_PL4L,0.00000,356.0,5067.0,0.000000,0.0,0.0
3,201611,Central Europe Market_PL4L,0.00000,791.0,3022.0,0.000000,0.0,0.0
4,201611,NorthWest Europe Market_PL4L,0.00000,115.0,2391.0,752.840420,0.0,0.0


In [8]:
#merge cost and reg
all_data = pd.merge(data, data_reg, how='left', on=['04 FISCAL MONTH', 'id'])
all_data.reset_index(inplace=True, drop=True)

In [9]:
all_data['unique_id'] = all_data['id'] + "_" + all_data['line_cost']

In [10]:
month_as_date_str = [str(x)[:4] + '-' + str(x)[4:] for x in all_data['04 FISCAL MONTH']]

In [11]:
month_date = pd.to_datetime(month_as_date_str)

In [12]:
dates = month_date.unique()
dtrain = dates[:-6]
dtest = dates[-6:]

In [13]:
all_data['ds'] = month_date

In [14]:
all_data = all_data[['unique_id',
             'ds',
             'y',
             'Net Revenues',
             'Warranty Units Adjusted',
             'WIB',
             'Total Number of Calls',
             'Total Number of Repair Events',
             'AIR (Avg)']]


In [15]:
train = all_data[all_data['ds'].isin(dtrain)].copy()
X_train = train.drop(columns=['y'])
y_train = train[['unique_id', 'ds', 'y']]

In [16]:
test = all_data[all_data['ds'].isin(dtest)].copy()
X_test = test.drop(columns=['y'])
y_test = test[['unique_id', 'ds', 'y']]

In [17]:
train.shape, test.shape

((1154307, 9), (104013, 9))

In [18]:
len(train['unique_id'].unique())

29799

In [19]:
len(test['unique_id'].unique())

20286

In [20]:
from statsforecast.models import AutoARIMA

In [21]:
models = [AutoARIMA(season_length = 7)]

sf = StatsForecast(
    models=models,
    freq='M',
    n_jobs=-1
)

In [22]:
horizon = 6
level = [95]

fcst = sf.forecast(df=train, h=horizon, X_df=X_test)
fcst = fcst.reset_index()
fcst.head()

ValueError: Expected X to have shape (178794, 7), but got (104013, 7)

In [162]:
fcst_u = sf.forecast(df=train[['unique_id', 'ds', 'y']], h=6, level=level)

ValueError: math domain error